<a href="https://colab.research.google.com/github/Computer-CGuy/BitcoinTPU/blob/main/Old_Songs_%7C_JaxEdition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import jax
import jax.numpy as np
import jax.numpy as jnp

In [2]:
from jax import grad, jit, vmap
from jax import random

In [3]:
from jax import device_put

## VMAP test

In [28]:
key = random.PRNGKey(0)
mat = random.normal(key, (150, 100))
batched_x = random.normal(key, (10, 100))


In [29]:
# key =100008

def apply_matrix(v):
  return jnp.dot(mat, v)+jnp.dot(mat, v)

In [30]:
def naively_batched_apply_matrix(v_batched):
  return jnp.stack([apply_matrix(v) for v in v_batched])

print('Naively batched')
%timeit naively_batched_apply_matrix(batched_x).block_until_ready()

Naively batched
100 loops, best of 5: 13.5 ms per loop


In [31]:
@jit
def batched_apply_matrix(v_batched):
  return jnp.dot(v_batched, mat.T)

print('Manually batched')
%timeit batched_apply_matrix(batched_x).block_until_ready()

Manually batched
The slowest run took 284.39 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 5: 62.6 µs per loop


In [32]:
@jit
def vmap_batched_apply_matrix(v_batched):
  return vmap(apply_matrix)(v_batched)

print('Auto-vectorized with vmap')
%timeit vmap_batched_apply_matrix(batched_x).block_until_ready()

Auto-vectorized with vmap
The slowest run took 831.21 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 5: 81.4 µs per loop


## Main Code


In [37]:
_k = [0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2]
w_init = [1633779712, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 1633779712, 655360, 4028130556, 1073742468, 1054728214, 16951296, 1275490269, 1650573323, 4195356184, 3358894600, 1190258893, 4283781196, 3396427740, 1310106865, 109912110, 3992192224, 3718962921, 3366911842, 606963060, 4270064082, 2838786825, 2976635877, 1549026422, 64942781, 2406927427, 2077005080, 3607598593, 2011791591, 4164220569, 2947864758, 3446355188, 4102404423, 3115010575, 2566757838, 584613669, 2397578555, 3038678904, 2334119052, 2922065041, 4099942322, 925755302, 2995116319, 1430589485, 2514036726, 4225725885, 1376483090, 357258090, 3690550341]

In [4]:
_h = [0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19]
def _rotr(x, y):
    return ((x >> y) | (x << (32 - y))) 
def _maj(x, y, z):
    return (x & y) ^ (x & z) ^ (y & z)
def _ch(x, y, z):
    return (x & y) ^ ((~x) & z)
def sigma0(x):
	return _rotr(x, 2) ^ _rotr(x, 13) ^ _rotr(x, 22)
def sigma1(x):
	return _rotr(x, 6) ^ _rotr(x, 11) ^ _rotr(x, 25)

In [24]:
F32 = 0xFFFFFFFF

In [5]:
def body_fun(i,sampled):
    # print(val)
    # w,a,b,c,d,e,f,g,h,k,i = val
    # a,b,c,d,e,f,g,h
    # 0,1,2,3,4,5,6,7
    s0 = sigma0(sampled[0])
    s1 = sigma1(sampled[4])


    t1 = sampled[7] + s1 + _ch(sampled[4], sampled[5], sampled[6])
    # k[i]+w[i]
    t2 = s0 + _maj(sampled[0], sampled[1], sampled[2]) 

    jax.ops.index_update(sampled, jax.ops.index[1:], sampled[:-1])
    jax.ops.index_update(sampled, jax.ops.index[0], t1+t2)
    jax.ops.index_update(sampled, jax.ops.index[3], (sampled[3] + t1) )
    # sampled[0]=t1+t2
    # sampled[4]=(sampled[3] + t1) 
    # h = g
    # g = f
    # f = e
    # e = (sampled[3] + t1) 
    # d = c
    # c = b	
    # b = a
    # a = (t1 + t2)
    return sampled

In [6]:
import random

In [6]:
import numpy as npo

In [7]:
# init_val = [random.randint(3, F32//10) for x in range(11)]
# # _h = [0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19]
# # init_val[1:-1]=_h
# init_val[0]=[random.randint(3, F32//10) for x in range(len(w_init))]
# init_val[-2]=[random.randint(3, F32//10) for x in range(len(_k))]
# init_val[-1]=0
key = random.PRNGKey(0)

In [ ]:
init_val

In [13]:
import timeit

In [10]:
# init_val = 0
start = 0
stop = 64

# in_array_size = 1000

# init_val = npo.random.randint(low=0,high=1000,size=(8,in_array_size),dtype=npo.int32)
# init_val = device_put(init_val)

In [48]:
def call():
    jax.lax.fori_loop(start, stop, body_fun, init_val).block_until_ready()

In [ ]:
timeit.timeit(call)

In [15]:
# in_array_size = 1000
%timeit jax.lax.fori_loop(start, stop, body_fun, init_val).block_until_ready()

The slowest run took 204.00 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 5: 815 µs per loop


In [8]:
import time


In [11]:
del init_val

In [22]:
def callme(init_val):
    return jax.lax.fori_loop(0, 64, body_fun, init_val)#.block_until_ready()

In [19]:
callme = jit(callme)

In [ ]:
# in_array_size = 1000
in_array_size = 50000000

init_val = npo.random.randint(low=0,high=1000,size=(8,in_array_size),dtype=npo.int32)
init_val = device_put(init_val)
t0 = time.time()

jax.vmap(callme,in_axes=1)(init_val).block_until_ready()
t1 = time.time()
print(("{:,}".format(in_array_size/(t1-t0))))
# print()

In [24]:
jax.vmap(callme)(init_val)

DeviceArray([[734, 456, 616, ..., 864, 806,   2],
             [170, 314, 126, ...,  82, 925, 835],
             [307, 978, 753, ..., 258, 702, 739],
             ...,
             [695, 959, 679, ..., 221, 209, 213],
             [138, 509, 477, ..., 370, 679, 885],
             [448, 718, 922, ..., 174, 967, 711]], dtype=int32)